In [ ]:
# Run in Colab GPU instance (e.g. T4) as it is needed to load models fitted with a GPU (NP bug?)
!pip uninstall -y torch notebook notebook_shim tensorflow tensorflow-datasets prophet torchaudio torchdata torchtext torchvision
!pip install git+https://github.com/ourownstory/neural_prophet.git

In [ ]:
#To check status of GPUs
!nvidia-smi

In [ ]:
import logging
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from matplotlib import gridspec
import pickle
import pandas as pd
import pickle
import os
import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm.auto import tqdm

# from prophet import Prophet
from neuralprophet import NeuralProphet, set_log_level

set_log_level("ERROR")
logging.getLogger("prophet").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")


# MASE:  Mean Absolute Scaled Error compares the mean absolute error of the forecast with
# the mean absolute error of a naive forecast, i.e. the forecast that simply repeats the last observed value.
def mase(y_test,y_pred,y_train):
    return (y_test-y_pred.reset_index(drop=True)).abs().mean()/y_train.diff().dropna().abs().mean()


In [ ]:
# Connecting to Google Drive and mount project to running instance

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Need to manually set project path.
fpath='/content/drive/MyDrive/Colab Notebooks/Capstone'

print(f"Project filepath, fpath: {fpath}")


# Load cleaned and reformated sales data
with open(fpath+'/data/df_v_m2.pkl', 'rb') as f:
    ddict=pickle.load(f)
    df_v_m=ddict['df_v_m']
    df_v_m_test=ddict['df_v_m_test']
# read price data
df_p_m=pd.read_pickle(fpath+'/data/df_flat_pvp.pkl')
df_p_m_test=pd.read_pickle(fpath+'/data/df_flat_pvp_test.pkl')
df_p_m_test["MED_GOA"]=df_p_m_test.loc[:,[c for c in df_p_m_test.columns if 'GOA' in c]].median(axis=1)
df_p_m_test["MED_95"]=df_p_m_test.loc[:,[c for c in df_p_m_test.columns if '95' in c]].median(axis=1)

# nearest station info for each station (3&8 have same prices so use 2nd nearest for them)
stn_near=pd.read_pickle(fpath+'/data/stn_near.pkl')

# read holidays defined in data and test period
df_hol_base=pd.read_pickle(fpath+'/data/df_hol_base.pkl')

# add extra holidays/events to the base holidays
# Neuralprophet uses only 'event' not both 'holiday' and 'event' as prophet
# Defined as days that appear repeatedly in the time series as low sales days but are not official holidays
# are assumed to be foreseeable events that can be used to improve the forecast
with open(fpath+'/data/df_holi_extra.pkl', 'rb') as f:
    dict_hol_extra=pickle.load(f)


# dicts to store model performance statistics for later comparison
dict_MASE={}
dict_MAE={}

# Model names
model_names={1:'Seasonal\nNeuralProphet',2:'Seasonal+AR',3:'Seasonal\n+AR-Net',4:'Seasonal\n+AR-Net+Meteo',
             5:'Seasonal\n+AR+$\Delta$y',6:'Seasonal+AR+\n$\Delta$y+Meteo',
             7:'Seasonal+AR+\n$\Delta$y+Meteo\n+$\Delta$PVP',8:'Seasonal+AR+\n$\Delta$y+Meteo\n+$\Delta$PVP+nearby $\Delta$PVP'}

# Function to draw the resulting forecast compared to test data
def plt_mth(df_mth,stn,model):

    fig,ax=plt.subplots(1,1,figsize=(12,8))
    ax.plot(df_mth.index,df_mth.y,label='Test',color='darkred',lw=2)
    ax.plot(df_mth.index,df_mth.yhatn,label='Forecast',color='darkblue',ls='--',lw=2)
    ax.fill_between(df_mth.index,df_mth['yhat lower'],df_mth['yhat upper'],color='lightblue',alpha=0.5)
    ax.grid('both')
    ax.legend()
    ax.set_ylabel('Sales')
    errorpct=(df_mth.yhatn.sum()-df_mth.y.sum())/df_mth.y.sum()
    ax.set_title(f"Forecast for {stn} using model {model_names[model]} Error: {errorpct*100:.1f}%")
    return fig,ax



# 1. Seasonalities only

In [ ]:
import os

dict_M = {}
valid = False
modelnr=1
fit_model=False # True to fit model, False to load fitted model from disk

fpathm = fpath+f"/m31_{modelnr}"
if not os.path.exists(fpathm):
    os.mkdir(fpathm)


for stn in tqdm([f"ES{nr}_{pr}" for nr in range(1,13) for pr in ["95","GOA"]]):

    # no extra holidays for ALL or 95 sales
    if stn[:2]=='ES' and stn[-3:]=="GOA": # seeing the effects of extra holidays if it improves the forecast and worth the effort
        df_hol_extra=dict_hol_extra[f"df_holi_extra_{stn}"]
        if df_hol_extra.loc[df_hol_extra.ds==datetime.datetime(2018,1,5)].shape[0]==1: #add extra event for 2019-01-04 if 2018-01-05 is present
            df_hol_extra=pd.concat([df_hol_extra,pd.DataFrame({'ds':[datetime.datetime(2019,1,4)],'event':['Ev01']},index=[0])],ignore_index=True)
        df_hol=pd.concat([df_hol_base,df_hol_extra],ignore_index=True)
        print(f"Added {df_hol_extra.shape[0]} extra holidays for",stn)
    else:
        df_hol=df_hol_base

    df=df_v_m.loc[:,[stn]].reset_index().rename(columns={'sale_date':'ds',stn:'y'})
    df_test=df_v_m_test.loc[:,[stn]].reset_index().rename(columns={'sale_date':'ds',stn:'y'})


    m=NeuralProphet(
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=False,
                    n_forecasts=31,
                    quantiles=[0.05,0.95],
                    epochs=60,
                    )

    for e in df_hol.event.unique():
      m.add_events(e,regularization=None,mode='additive')#,lower_window=0,upper_window=1)

    df_new=m.create_df_with_events(df, df_hol)

    if fit_model:
        if valid:
            df_train, df_val = m.split_df(df_new, freq="D", valid_p=0.1)
            metrics=m.fit(df_train,freq='D',validation_df=df_val, progress='bar')
        else:
            df_train=df_new
            metrics=m.fit(df_train,freq='D', progress='bar')
        print(metrics.tail(1))

        # save fitted model to disk
        with open(fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl', "wb") as f:
            pickle.dump(m, f, pickle.HIGHEST_PROTOCOL)
    else:
        print("Loading model from file: "+fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl')
        with open(fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl', "rb") as f:
            m=pickle.load(f)


    # forecast_train = m.predict(df_new)
    m.set_plotting_backend("matplotlib")


    # one forecast on '2018-12-31' for 31 days ahead (2019-01-01 to 2019-01-31)
    # in seasonal forecasting the model does not need own forecast to roll the model forward
    # so we can use the same model to forecast the next 31 days else we ask the model to roll the model forward 31 days
    if m.model.n_forecasts==1:
      future = m.make_future_dataframe(m.create_df_with_events(pd.concat([df]),df_hol),events_df=df_hol,
                                  periods=31, n_historic_predictions=31*1)
      forecast_test = m.predict(future)
      f31d=forecast_test.loc[forecast_test.ds>'2018-12-31','ds':'trend']
      f31d.set_index('ds',inplace=True)
      f31d.rename(columns={'yhat1':'yhatn'},inplace=True)
      f31d['y']=df_test.y.values

    else: # rolling the model forward 31 days on the first day

      future = m.make_future_dataframe(m.create_df_with_events(pd.concat([df]),df_hol),events_df=df_hol,
                                    periods=1, n_historic_predictions=31*1)
      forecast_test = m.predict(future)

      f31d=forecast_test.loc[forecast_test.ds=='2018-12-31',[f"yhat{i}" for i in range(1,32)]].T
      f31_5=forecast_test.loc[forecast_test.ds=='2018-12-31',[c for c in forecast_test.columns if ' 5.0%' in c]].T
      f31_95=forecast_test.loc[forecast_test.ds=='2018-12-31',[c for c in forecast_test.columns if ' 95.0%' in c]].T
      f31d['yhat upper']=f31_95.values
      f31d['yhat lower']=f31_5.values
      f31d['ds']=df_test.ds.values
      f31d.set_index('ds',inplace=True)
      f31d.rename(columns={f31d.columns[0]:'yhatn'},inplace=True)
      f31d['y']=df_test.y.values

    f31d.to_pickle(fpathm+f"/{stn}_model_{modelnr}_f31d.pkl")

    # Plot 31d forecast vs. test data
    # fig,ax=plt.subplots(figsize=(10,6))
    # f31d.plot(ax=ax,grid=True,lw=2,title=f"{stn} model {modelnr} sales 31d forecast vs. test data",xlabel="Date",ylabel=f"{stn} sales")
    fig,ax=plt_mth(f31d.rename(columns={'yhat1 5.0%':'yhat lower','yhat1 95.0%':'yhat upper'}),stn,modelnr)
    fig.savefig(fpathm+f"/{stn}_{modelnr}_31d_sales_forecast.png",bbox_inches='tight')

    print(f"Model {modelnr}: {stn} 31d forecast {f31d.yhatn.sum():.2f} vs. test data {f31d.y.sum():.2f}.  Error: {-(f31d.y.sum()-f31d.yhatn.sum())/f31d.y.sum()*100:.1f}%")


    plt.close()



# 2. One month (31d) ahead forecast with Auto-Regression

In [ ]:
valid=False
dict_M = {}
modelnr=2
fit_model=False

fpathm = fpath+f"/m31_{modelnr}"
if not os.path.exists(fpathm):
    os.mkdir(fpathm)

for stn in tqdm([f"ES{nr}_{pr}" for nr in range(1,13) for pr in ["95","GOA"]]):

    # no extra holidays for ALL or 95 sales
    if stn[:2]=='ES' and stn[-3:]=="GOA":
        df_hol_extra=dict_hol_extra[f"df_holi_extra_{stn}"]
        if df_hol_extra.loc[df_hol_extra.ds==datetime.datetime(2018,1,5)].shape[0]==1: #add extra event for 2019-01-04 if 2018-01-05 is present
            df_hol_extra=pd.concat([df_hol_extra,pd.DataFrame({'ds':[datetime.datetime(2019,1,4)],'event':['Ev01']},index=[0])],ignore_index=True)
        df_hol=pd.concat([df_hol_base,df_hol_extra],ignore_index=True)
        print(f"Added {df_hol_extra.shape[0]} extra holidays for",stn)
    else:
        df_hol=df_hol_base


    df=df_v_m.loc[:,[stn]].reset_index().rename(columns={'sale_date':'ds',stn:'y'})
    df_test=df_v_m_test.loc[:,[stn]].reset_index().rename(columns={'sale_date':'ds',stn:'y'})


    m=NeuralProphet(
                yearly_seasonality=True,
                weekly_seasonality=True,
                daily_seasonality=False,
                seasonality_mode="additive",
                n_lags=7,
                ar_reg=.1,
                n_forecasts=31,
                quantiles=[0.05,0.95],
                #epochs=60,
                )

    for e in df_hol.event.unique():
      m.add_events(e,regularization=None,mode='additive')#,lower_window=0,upper_window=1)

    df_new=m.create_df_with_events(df, df_hol)

    if fit_model:
        if valid:
            df_train, df_val = m.split_df(df_new, freq="D", valid_p=0.1)
            metrics=m.fit(df_train,freq='D',validation_df=df_val, progress='bar')
        else:
            df_train=df_new
            metrics=m.fit(df_train,freq='D', progress='bar')
        print(metrics.tail(1))

        # save fitted model to disk
        with open(fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl', "wb") as f:
            pickle.dump(m, f, pickle.HIGHEST_PROTOCOL)
    else:
        print("Loading model from file: "+fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl')
        with open(fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl', "rb") as f:
            m=pickle.load(f)



    m.set_plotting_backend("matplotlib")



    # one forecast on '2018-12-31' for 31 days ahead (2019-01-01 to 2019-01-31)
    # in seasonal forecasting the model does not need own forecast to roll the model forward
    # so we can use the same model to forecast the next 31 days else we ask the model to roll the model forward 31 days
    if m.model.n_forecasts==1:
      future = m.make_future_dataframe(m.create_df_with_events(pd.concat([df]),df_hol),events_df=df_hol,
                                  periods=31, n_historic_predictions=31*1)
      forecast_test = m.predict(future)
      f31d=forecast_test.loc[forecast_test.ds>'2018-12-31','ds':'trend']
      f31d.set_index('ds',inplace=True)
      f31d.rename(columns={'yhat1':'yhatn'},inplace=True)
      f31d['y']=df_test.y.values
    else: # rolling the model forward 31 days on the first day

      future = m.make_future_dataframe(m.create_df_with_events(pd.concat([df]),df_hol), events_df=df_hol,
                                periods=31, n_historic_predictions=31*1)

      forecast_test = m.predict(future,raw=True)
      f31d=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and '%' not in c]].T
      f31_5=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and ' 5.0%' in c]].T
      f31_95=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and ' 95.0%' in c]].T

      f31d['yhat upper']=f31_95.values
      f31d['yhat lower']=f31_5.values
      f31d['ds']=df_test.ds.values
      f31d.set_index('ds',inplace=True)
      f31d.rename(columns={f31d.columns[0]:'yhatn'},inplace=True)
      f31d['y']=df_test.y.values

    f31d.to_pickle(fpathm+f"/{stn}_model_{modelnr}_f31d.pkl")

    # Plot 31d forecast vs. test data
    # fig,ax=plt.subplots(figsize=(10,6))
    # f31d.plot(ax=ax,grid=True,lw=2,title=f"{stn} model {modelnr} sales 31d forecast vs. test data",xlabel="Date",ylabel=f"{stn} sales")
    fig,ax=plt_mth(f31d.rename(columns={'yhat1 5.0%':'yhat lower','yhat1 95.0%':'yhat upper'}),stn,modelnr)
    fig.savefig(fpathm+f"/{stn}_{modelnr}_31d_sales_forecast.png",bbox_inches='tight')

    print(f"Model {modelnr}: {stn} 31d forecast {f31d.yhatn.sum():.2f} vs. test data {f31d.y.sum():.2f}.  Error: {-(f31d.y.sum()-f31d.yhatn.sum())/f31d.y.sum()*100:.1f}%")

    plt.close()

# 3 One month (31d) ahead forecast modeling AR with a Neural Network (AR-Net)

In [ ]:
valid=False
dict_M = {}
modelnr=3
fit_model=False

fpathm = fpath+f"/m31_{modelnr}"
if not os.path.exists(fpathm):
    os.mkdir(fpathm)

for stn in tqdm([f"ES{nr}_{pr}" for nr in range(1,13) for pr in ["95","GOA"]]):

    if stn[:2]=='ES' and stn[-3:]=="GOA":
        df_hol_extra=dict_hol_extra[f"df_holi_extra_{stn}"]
        df_hol=pd.concat([df_hol_base,df_hol_extra],ignore_index=True)
        print(f"Added {df_hol_extra.shape[0]} extra holidays for",stn)
    else:
        df_hol=df_hol_base

    df=df_v_m.loc[:,[stn]].reset_index().rename(columns={'sale_date':'ds',stn:'y'})
    df_test=df_v_m_test.loc[:,[stn]].reset_index().rename(columns={'sale_date':'ds',stn:'y'})

    nl=8
    valid=False

    m=NeuralProphet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            seasonality_mode="additive",
            n_lags=7,
            n_forecasts=31,
            quantiles=[0.05,0.95],
            ar_layers=[nl, nl, nl, nl],
            ar_reg=10,
            # learning_rate=0.003,
            )

    for e in df_hol.event.unique():
      m.add_events(e,regularization=None,mode='additive')#,lower_window=0,upper_window=1)

    df_new=m.create_df_with_events(df, df_hol)

    if fit_model:
        if valid:
            df_train, df_val = m.split_df(df_new, freq="D", valid_p=0.1)
            metrics=m.fit(df_train,freq='D',validation_df=df_val, progress='bar')
        else:
            df_train=df_new
            metrics=m.fit(df_train,freq='D', progress='bar')
        print(metrics.tail(1))

        # save fitted model to disk
        with open(fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl', "wb") as f:
            pickle.dump(m, f, pickle.HIGHEST_PROTOCOL)
    else:
        print("Loading model from file: "+fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl')
        with open(fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl', "rb") as f:
            m=pickle.load(f)




    m.set_plotting_backend("matplotlib")


    # one forecast on '2018-12-31' for 31 days ahead (2019-01-01 to 2019-01-31)
    # in seasonal forecasting the model does not need own forecast to roll the model forward
    # so we can use the same model to forecast the next 31 days else we ask the model to roll the model forward 31 days
    if m.model.n_forecasts==1:
      future = m.make_future_dataframe(m.create_df_with_events(pd.concat([df]),df_hol),events_df=df_hol,
                                  periods=31, n_historic_predictions=31*1)
      forecast_test = m.predict(future)
      f31d=forecast_test.loc[forecast_test.ds>'2018-12-31','ds':'trend']
      f31d.set_index('ds',inplace=True)
      f31d.rename(columns={'yhat1':'yhatn'},inplace=True)
      f31d['y']=df_test.y.values

    else: # rolling the model forward 31 days on the first day

      future = m.make_future_dataframe(m.create_df_with_events(pd.concat([df]),df_hol), events_df=df_hol,
                                periods=31, n_historic_predictions=31*1)

      forecast_test = m.predict(future,raw=True)
      f31d=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and '%' not in c]].T
      f31_5=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and ' 5.0%' in c]].T
      f31_95=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and ' 95.0%' in c]].T

      f31d['yhat upper']=f31_95.values
      f31d['yhat lower']=f31_5.values
      f31d['ds']=df_test.ds.values
      f31d.set_index('ds',inplace=True)
      f31d.rename(columns={f31d.columns[0]:'yhatn'},inplace=True)
      f31d['y']=df_test.y.values


    f31d.to_pickle(fpathm+f"/{stn}_model_{modelnr}_f31d.pkl")

    # Plot 31d forecast vs. test data
    fig,ax=plt_mth(f31d.rename(columns={'yhat1 5.0%':'yhat lower','yhat1 95.0%':'yhat upper'}),stn,modelnr)
    fig.savefig(fpathm+f"/{stn}_{modelnr}_31d_sales_forecast.png",bbox_inches='tight')

    print(f"Model {modelnr}: {stn} 31d forecast {f31d.yhatn.sum():.2f} vs. test data {f31d.y.sum():.2f}.  Error: {-(f31d.y.sum()-f31d.yhatn.sum())/f31d.y.sum()*100:.1f}%")


    plt.close()


# 4 AR-Net + meteo

In [ ]:
dict_M={}
valid=False
modelnr=4
fit_model=False

fpathm = fpath+f"/m31_{modelnr}"
if not os.path.exists(fpathm):
    os.mkdir(fpathm)

for stn in tqdm([f"ES{nr}_{pr}" for nr in range(1,13) for pr in ["95","GOA"]]):

    if stn[:2]=='ES' and stn[-3:]=="GOA":
        df_hol_extra=dict_hol_extra[f"df_holi_extra_{stn}"]
        if df_hol_extra.loc[df_hol_extra.ds==datetime.datetime(2018,1,5)].shape[0]==1: #add extra event for 2019-01-04 if 2018-01-05 is present
            df_hol_extra=pd.concat([df_hol_extra,pd.DataFrame({'ds':[datetime.datetime(2019,1,4)],'event':['Ev01']},index=[0])],ignore_index=True)
        df_hol=pd.concat([df_hol_base,df_hol_extra],ignore_index=True)
        print(f"Added {df_hol_extra.shape[0]} extra holidays for",stn)
    else:
        df_hol=df_hol_base

    df=df_v_m.loc[:,[stn]].reset_index().rename(columns={'sale_date':'ds',stn:'y'})
    df_test=df_v_m_test.loc[:,[stn]].reset_index().rename(columns={'sale_date':'ds',stn:'y'})

    nl=8
    valid=False

    # add weather data, temp, precipitation, wind
    df["tmed"] = df_v_m.tmed.values
    df["prec"] = df_v_m.prec.values
    df["velmedia"] = df_v_m.velmedia.values
    df["sol"] = df_v_m.sol.values
    df["racha"] = df_v_m.racha.values

    df_test["tmed"] = df_v_m_test.tmed.values
    df_test["prec"] = df_v_m_test.prec.values
    df_test["velmedia"] = df_v_m_test.velmedia.values
    df_test["sol"] = df_v_m_test.sol.values
    df_test["racha"]= df_v_m_test.racha.values

    m=NeuralProphet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            seasonality_mode="additive",
            n_lags=7,
            n_forecasts=31,
            quantiles=[0.05,0.95],
            ar_layers=[nl, nl, nl, nl],
            ar_reg=10,
            # learning_rate=0.003,
            )

    for e in df_hol.event.unique():
      m.add_events(e,regularization=None,mode='additive')#,lower_window=0,upper_window=1)

    # add weather data as future regressors and assume reliable weather forecast is available for 1 day ahead
    m.add_future_regressor("tmed",normalize="standardize")
    m.add_future_regressor("prec",normalize="standardize")
    m.add_future_regressor("velmedia",normalize="standardize")
    m.add_future_regressor("sol",normalize="standardize")
    m.add_future_regressor("racha",normalize="standardize")


    df_new=m.create_df_with_events(df, df_hol)
    m.set_plotting_backend("matplotlib")

    if fit_model:
        if valid:
            df_train, df_val = m.split_df(df_new, freq="D", valid_p=0.1)
            metrics=m.fit(df_train,freq='D',validation_df=df_val, progress='bar')
        else:
            df_train=df_new
            metrics=m.fit(df_train,freq='D', progress='bar')
        print(metrics.tail(1))

        # save fitted model to disk
        with open(fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl', "wb") as f:
            pickle.dump(m, f, pickle.HIGHEST_PROTOCOL)
    else:
        print("Loading model from file: "+fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl')
        with open(fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl', "rb") as f:
            m=pickle.load(f)

    m.set_plotting_backend("matplotlib")


    regressors_list=['tmed','prec','velmedia','sol','racha',]

    # one forecast on '2018-12-31' for 31 days ahead (2019-01-01 to 2019-01-31)
    # in seasonal forecasting the model does not need own forecast to roll the model forward
    # so we can use the same model to forecast the next 31 days else we ask the model to roll the model forward 31 days
    if m.model.n_forecasts==1:
      future = m.make_future_dataframe(m.create_df_with_events(pd.concat([df,df_test]),df_hol),
                                    regressors_df=m.create_df_with_events(pd.concat([df,df_test]),df_hol).loc[:,regressors_list],
                                    periods=31, n_historic_predictions=31*1)
      forecast_test = m.predict(future)
      f31d=forecast_test.loc[forecast_test.ds>'2018-12-31','ds':'trend']
      f31d.set_index('ds',inplace=True)
      f31d.rename(columns={'yhat1':'yhatn'},inplace=True)
      f31d['y']=df_test.y.values
    else: # rolling the model forward 31 days on the first day
      future = m.make_future_dataframe(m.create_df_with_events(pd.concat([df]),df_hol), events_df=df_hol,
                                regressors_df=pd.concat([df,df_test]).loc[:,regressors_list],
                                periods=31, n_historic_predictions=31*1)

      forecast_test = m.predict(future,raw=True)
      f31d=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and '%' not in c]].T
      f31_5=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and ' 5.0%' in c]].T
      f31_95=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and ' 95.0%' in c]].T

      f31d['yhat upper']=f31_95.values
      f31d['yhat lower']=f31_5.values
      f31d['ds']=df_test.ds.values
      f31d.set_index('ds',inplace=True)
      f31d.rename(columns={f31d.columns[0]:'yhatn'},inplace=True)
      f31d['y']=df_test.y.values

    f31d.to_pickle(fpathm+f"/{stn}_model_{modelnr}_f31d.pkl")

    # Plot 31d forecast vs. test data
    fig,ax=plt_mth(f31d.rename(columns={'yhat1 5.0%':'yhat lower','yhat1 95.0%':'yhat upper'}),stn,modelnr)
    fig.savefig(fpathm+f"/{stn}_{modelnr}_31d_sales_forecast.png",bbox_inches='tight')

    print(f"Model {modelnr}: {stn} 31d forecast {f31d.yhatn.sum():.2f} vs. test data {f31d.y.sum():.2f}.  Error: {-(f31d.y.sum()-f31d.yhatn.sum())/f31d.y.sum()*100:.1f}%")

    plt.close()

# 5 One (1) step ahead forecast using Auto-Regression $+\Delta y$ lagged regressor

In [ ]:
valid=False
dict_M={}
modelnr=5
fit_model=False

fpathm = fpath+f"/m31_{modelnr}"
if not os.path.exists(fpathm):
    os.mkdir(fpathm)

for stn in tqdm([f"ES{nr}_{pr}" for nr in range(1,13) for pr in ["95","GOA"]]):

    # no extra holidays for ALL or 95 sales
    if stn[:2]=='ES' and stn[-3:]=="GOA":
        df_hol_extra=dict_hol_extra[f"df_holi_extra_{stn}"]
        if df_hol_extra.loc[df_hol_extra.ds==datetime.datetime(2018,1,5)].shape[0]==1: #add extra event for 2019-01-04 if 2018-01-05 is present
            df_hol_extra=pd.concat([df_hol_extra,pd.DataFrame({'ds':[datetime.datetime(2019,1,4)],'event':['Ev01']},index=[0])],ignore_index=True)
        df_hol=pd.concat([df_hol_base,df_hol_extra],ignore_index=True)
        print(f"Added {df_hol_extra.shape[0]} extra holidays for",stn)
    else:
        df_hol=df_hol_base


    df=df_v_m.loc[:,[stn]].reset_index().rename(columns={'sale_date':'ds',stn:'y'})
    df_test=df_v_m_test.loc[:,[stn]].reset_index().rename(columns={'sale_date':'ds',stn:'y'})


    lagg_I=pd.concat([df,df_test],ignore_index=True).y.diff().fillna(0)

    #df2=df.copy(deep=True)
    df["I"] = lagg_I.iloc[:len(df)].values

    #df_test2=df_test.copy(deep=True)
    df_test["I"]=lagg_I.iloc[len(df):].values


    m=NeuralProphet(
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=False,
                    seasonality_mode="additive",
                    n_lags=7,
                    ar_reg=1,
                    n_forecasts=31,
                    quantiles=[0.05,0.95],
                    #epochs=60,
                    )

    m.add_lagged_regressor("I",normalize="standardize",n_lags=7)


    for e in df_hol.event.unique():
      m.add_events(e,regularization=None,mode='additive')#,lower_window=0,upper_window=1)

    df_new=m.create_df_with_events(df, df_hol)

    if fit_model:
       if valid:
          df_train, df_val = m.split_df(df_new, freq="D", valid_p=0.1)
          metrics=m.fit(df_train,freq='D',validation_df=df_val, progress='bar')
       else:
          df_train=df_new
          metrics=m.fit(df_train,freq='D', progress='bar')
       print(metrics.tail(1))

      # save fitted model to disk
       with open(fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl', "wb") as f:
          pickle.dump(m, f, pickle.HIGHEST_PROTOCOL)
    else:
        print("Loading model from file: "+fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl')
        with open(fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl', "rb") as f:
          m=pickle.load(f)


    m.set_plotting_backend("matplotlib")


    # one forecast on '2018-12-31' for 31 days ahead (2019-01-01 to 2019-01-31)
    # in seasonal forecasting the model does not need own forecast to roll the model forward
    # so we can use the same model to forecast the next 31 days else we ask the model to roll the model forward 31 days
    if m.model.n_forecasts==1:
      print("One forecast!")
      future = m.make_future_dataframe(m.create_df_with_events(pd.concat([df,df_test]),df_hol), events_df=df_hol,
                        regressors_df=pd.DataFrame({'ds':np.append(df_new.ds.values,df_test.ds.values),'I':lagg_I.values}).reset_index(drop=True),
                                     periods=31, n_historic_predictions=31*4)
      forecast_test = m.predict(future)
      f31d=forecast_test.loc[forecast_test.ds>'2018-12-31','ds':'trend']
      f31d.set_index('ds',inplace=True)
      f31d.rename(columns={'yhat1':'yhatn'},inplace=True)
      f31d['y']=df_test.y.values
    else: # rolling the model forward 31 days on the first day

      future = m.make_future_dataframe(m.create_df_with_events(pd.concat([df]),df_hol), events_df=df_hol,
                        regressors_df=pd.DataFrame({'ds':np.append(df_new.ds.values,df_test.ds.values),'I':lagg_I.values}).reset_index(drop=True),
                                periods=31, n_historic_predictions=31*1)

      forecast_test = m.predict(future,raw=True)
      f31d=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and '%' not in c]].T
      f31_5=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and ' 5.0%' in c]].T
      f31_95=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and ' 95.0%' in c]].T

      f31d['yhat upper']=f31_95.values
      f31d['yhat lower']=f31_5.values
      f31d['ds']=df_test.ds.values
      f31d.set_index('ds',inplace=True)
      f31d.rename(columns={f31d.columns[0]:'yhatn'},inplace=True)
      f31d['y']=df_test.y.values

    f31d.to_pickle(fpathm+f"/{stn}_model_{modelnr}_f31d.pkl")

    # Plot 31d forecast vs. test data
    fig,ax=plt_mth(f31d.rename(columns={'yhat1 5.0%':'yhat lower','yhat1 95.0%':'yhat upper'}),stn,modelnr)
    fig.savefig(fpathm+f"/{stn}_{modelnr}_31d_sales_forecast.png",bbox_inches='tight')

    print(f"Model {modelnr}: {stn} 31d forecast {f31d.yhatn.sum():.2f} vs. test data {f31d.y.sum():.2f}.  Error: {-(f31d.y.sum()-f31d.yhatn.sum())/f31d.y.sum()*100:.1f}%")

    plt.close()

# 6 One month (31d) step ahead forecast using AR, $+\Delta y$ lagged regressor and meteo

In [ ]:
dict_M={}
valid=False

modelnr=6
fit_model=False

fpathm = fpath+f"/m31_{modelnr}"
if not os.path.exists(fpathm):
    os.mkdir(fpathm)


for stn in tqdm([f"ES{nr}_{pr}" for nr in range(1,13) for pr in ["95","GOA"]]):

    # add extra holidays/events
    # with open(fpath+'/df_holi_extra.pkl', 'rb') as f:
    #     dict_hol_extra=pickle.load(f)

    if stn[:2]=='ES' and stn[-3:]=="GOA":
        df_hol_extra=dict_hol_extra[f"df_holi_extra_{stn}"]
        df_hol_extra=pd.concat([df_hol_extra,pd.DataFrame({'ds':[datetime.datetime(2019,1,4)],'event':['Ev01']},index=[0])],ignore_index=True)
        df_hol=pd.concat([df_hol_base,df_hol_extra],ignore_index=True)
        print(f"Added {df_hol_extra.shape[0]} extra holidays for",stn)
        print(df_hol_extra)
    else:
        df_hol=df_hol_base

    df=df_v_m.loc[:,[stn]].reset_index().rename(columns={'sale_date':'ds',stn:'y'})
    df_test=df_v_m_test.loc[:,[stn]].reset_index().rename(columns={'sale_date':'ds',stn:'y'})

    # add lagged regressor with difference of y
    # do the difference on concatenated df and df_test for continuity
    lagg_I=pd.concat([df,df_test],ignore_index=True).y.diff().fillna(0)

    df["I"] = lagg_I.iloc[:len(df)].values
    df_test["I"]=lagg_I.iloc[len(df):].values

    # add weather data, temp, precipitation, wind
    # df["tmax"] = df_v_m.tmax.values
    df["tmed"] = df_v_m.tmed.values
    # df["tmin"] = df_v_m.tmin.values
    df["prec"] = df_v_m.prec.values
    df["velmedia"] = df_v_m.velmedia.values
    df["sol"] = df_v_m.sol.values
    df["racha"] = df_v_m.racha.values

    # df_test["tmax"] = df_v_m_test.tmax.values
    df_test["tmed"] = df_v_m_test.tmax.values
    # df_test["tmin"] = df_v_m_test.tmax.values
    df_test["prec"] = df_v_m_test.prec.values
    df_test["velmedia"] = df_v_m_test.velmedia.values
    df_test["sol"] = df_v_m_test.sol.values
    df_test["racha"] = df_v_m_test.racha.values

    m=NeuralProphet(
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=False,
                    seasonality_mode="additive",
                    n_lags=7,
                    ar_reg=0.1,
                    n_forecasts=31,
                    quantiles=[0.05,0.95],
                    #epochs=60,
                    )

    # add holidays
    for e in df_hol.event.unique():
      m.add_events(e,regularization=None,mode='additive')#,lower_window=0,upper_window=1)

    # add lagged regressor
    m.add_lagged_regressor("I",normalize="standardize",n_lags=7,regularization=0.1)

    # add weather data as future regressors and assume reliable weather forecast is available for 1 day ahead
    # m.add_future_regressor("tmax",normalize="standardize")
    m.add_future_regressor("tmed",normalize="standardize")
    # m.add_future_regressor("tmin",normalize="standardize")
    m.add_future_regressor("prec",normalize="standardize")
    m.add_future_regressor("velmedia",normalize="standardize")
    m.add_future_regressor("sol",normalize="standardize")
    m.add_future_regressor("racha",normalize="standardize")

    df_new=m.create_df_with_events(df, df_hol)


    if fit_model:
        if valid:
            df_train, df_val = m.split_df(df_new, freq="D", valid_p=0.1)
            metrics=m.fit(df_train,freq='D',validation_df=df_val, progress='bar')
        else:
            df_train=df_new
            metrics=m.fit(df_train,freq='D', progress='bar')
        print(metrics.tail(1))

        # save fitted model to disk
        with open(fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl', "wb") as f:
            pickle.dump(m, f, pickle.HIGHEST_PROTOCOL)
    else:
        print("Loading model from file: "+fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl')
        with open(fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl', "rb") as f:
            m=pickle.load(f)

    m.set_plotting_backend("matplotlib")

    # Forecast test period
    regressors_list=['I','tmed','prec','velmedia','sol','racha']


    # one forecast on '2018-12-31' for 31 days ahead (2019-01-01 to 2019-01-31)
    # in seasonal forecasting the model does not need own forecast to roll the model forward
    # so we can use the same model to forecast the next 31 days else we ask the model to roll the model forward 31 days
    if m.model.n_forecasts==1:
      print("One forecast!")
      future = m.make_future_dataframe(m.create_df_with_events(pd.concat([df,df_test]),df_hol), events_df=df_hol,
                                  regressors_df=pd.concat([df,df_test]).loc[:,regressors_list],
                                  periods=1, n_historic_predictions=31*1)

      forecast_test = m.predict(future)
      f31d=forecast_test.loc[forecast_test.ds>'2018-12-31','ds':'trend']
      f31d.set_index('ds',inplace=True)
      f31d.rename(columns={'yhat1':'yhatn'},inplace=True)

    else: # rolling the model forward 31 days on the first day
      future = m.make_future_dataframe(m.create_df_with_events(pd.concat([df]),df_hol), events_df=df_hol,
                                regressors_df=pd.concat([df]).loc[:,regressors_list],
                                periods=31, n_historic_predictions=31*1)

      forecast_test = m.predict(future,raw=True)
      f31d=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and '%' not in c]].T
      # f31d=forecast_test.loc[forecast_test.ds=='2018-12-31',[f"yhat{i}" for i in range(1,32)]].T
      f31_5=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and ' 5.0%' in c]].T
      # f31_5=forecast_test.loc[forecast_test.ds=='2018-12-31',[c for c in forecast_test.columns if ' 5.0%' in c]].T
      f31_95=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and ' 95.0%' in c]].T
      # f31_95=forecast_test.loc[forecast_test.ds=='2018-12-31',[c for c in forecast_test.columns if ' 95.0%' in c]].T

      f31d['yhat upper']=f31_95.values
      f31d['yhat lower']=f31_5.values
      f31d['ds']=df_test.ds.values
      f31d.set_index('ds',inplace=True)
      f31d.rename(columns={f31d.columns[0]:'yhatn'},inplace=True)
      f31d['y']=df_test.y.values

    f31d.to_pickle(fpathm+f"/{stn}_model_{modelnr}_f31d.pkl")

    # Plot 31d forecast vs. test data
    fig,ax=plt_mth(f31d.rename(columns={'yhat1 5.0%':'yhat lower','yhat1 95.0%':'yhat upper'}),stn,modelnr)
    fig.savefig(fpathm+f"/{stn}_{modelnr}_31d_sales_forecast.png",bbox_inches='tight')

    print(f"Model {modelnr}: {stn} 31d forecast {f31d.yhatn.sum():.2f} vs. test data {f31d.y.sum():.2f}.  Error: {-(f31d.y.sum()-f31d.yhatn.sum())/f31d.y.sum()*100:.1f}%")

    plt.close()

## 7. Model 6 with added PVP deviation from median as lagged regressor

In [ ]:
dict_M={}
valid=False
modelnr=7
fit_model=False

fpathm = fpath+f"/m31_{modelnr}"
if not os.path.exists(fpathm):
    os.mkdir(fpathm)

for stn in tqdm([f"ES{nr}_{pr}" for nr in range(1,13) for pr in ["95","GOA"]]):

    # add extra holidays/events
    # with open(fpath+'/df_holi_extra.pkl', 'rb') as f:
    #     dict_hol_extra=pickle.load(f)

    if stn[:2]=='ES' and stn[-3:]=="GOA":
        df_hol_extra=dict_hol_extra[f"df_holi_extra_{stn}"]
        df_hol_extra=pd.concat([df_hol_extra,pd.DataFrame({'ds':[datetime.datetime(2019,1,4)],'event':['Ev01']},index=[0])],ignore_index=True)
        df_hol=pd.concat([df_hol_base,df_hol_extra],ignore_index=True)
        print(f"Added {df_hol_extra.shape[0]} extra holidays for",stn)
        print(df_hol_extra)
    else:
        df_hol=df_hol_base

    df=df_v_m.loc[:,[stn]].reset_index().rename(columns={'sale_date':'ds',stn:'y'})
    df_test=df_v_m_test.loc[:,[stn]].reset_index().rename(columns={'sale_date':'ds',stn:'y'})

    # add lagged regressor with difference of y
    # do the difference on concatenated df and df_test for continuity
    lagg_I=pd.concat([df,df_test],ignore_index=True).y.diff().fillna(0)

    df["I"] = lagg_I.iloc[:len(df)].values
    df_test["I"]=lagg_I.iloc[len(df):].values

    # add weather data, temp, precipitation, wind
    # df["tmax"] = df_v_m.tmax.values
    df["tmed"] = df_v_m.tmed.values
    # df["tmin"] = df_v_m.tmin.values
    df["prec"] = df_v_m.prec.values
    df["velmedia"] = df_v_m.velmedia.values
    df["sol"] = df_v_m.sol.values
    df["racha"] = df_v_m.racha.values

    # add price deviation from median for this station
    df["pvpdev"]=(df_p_m[stn]-df_p_m["MED_"+stn.split('_')[1]]).values

    # df_test["tmax"] = df_v_m_test.tmax.values
    df_test["tmed"] = df_v_m_test.tmax.values
    # df_test["tmin"] = df_v_m_test.tmax.values
    df_test["prec"] = df_v_m_test.prec.values
    df_test["velmedia"] = df_v_m_test.velmedia.values
    df_test["sol"] = df_v_m_test.sol.values
    df_test["racha"] = df_v_m_test.racha.values

    # add price deviation from median for this station
    df_test["pvpdev"]=(df_p_m_test[stn]-df_p_m_test["MED_"+stn.split('_')[1]]).values

    m=NeuralProphet(
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=False,
                    seasonality_mode="additive",
                    n_lags=7,
                    ar_reg=0.1,
                    n_forecasts=31,
                    quantiles=[0.05,0.95],
                    #epochs=60,
                    )

    # add holidays
    for e in df_hol.event.unique():
      m.add_events(e,regularization=None,mode='additive')#,lower_window=0,upper_window=1)

    # add lagged regressor
    m.add_lagged_regressor("I",normalize="standardize",n_lags=7,regularization=0.1)
    m.add_lagged_regressor("pvpdev",normalize="standardize")

    # add weather data as future regressors and assume reliable weather forecast is available for 1 day ahead
    # m.add_future_regressor("tmax",normalize="standardize")
    m.add_future_regressor("tmed",normalize="standardize")
    # m.add_future_regressor("tmin",normalize="standardize")
    m.add_future_regressor("prec",normalize="standardize")
    m.add_future_regressor("velmedia",normalize="standardize")
    m.add_future_regressor("sol",normalize="standardize")
    m.add_future_regressor("racha",normalize="standardize")

    df_new=m.create_df_with_events(df, df_hol)


    if fit_model:
        if valid:
            df_train, df_val = m.split_df(df_new, freq="D", valid_p=0.1)
            metrics=m.fit(df_train,freq='D',validation_df=df_val, progress='bar')
        else:
            df_train=df_new
            metrics=m.fit(df_train,freq='D', progress='bar')
        print(metrics.tail(1))

        # save fitted model to disk
        with open(fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl', "wb") as f:
            pickle.dump(m, f, pickle.HIGHEST_PROTOCOL)
    else:
        print("Loading model from file: "+fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl')
        with open(fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl', "rb") as f:
            m=pickle.load(f)

    m.set_plotting_backend("matplotlib")

    # Forecast test period
    regressors_list=['I','pvpdev','tmed','prec','velmedia','sol','racha']


    # one forecast on '2018-12-31' for 31 days ahead (2019-01-01 to 2019-01-31)
    # in seasonal forecasting the model does not need own forecast to roll the model forward
    # so we can use the same model to forecast the next 31 days else we ask the model to roll the model forward 31 days
    if m.model.n_forecasts==1:
      print("One forecast!")
      future = m.make_future_dataframe(m.create_df_with_events(pd.concat([df,df_test]),df_hol), events_df=df_hol,
                                  regressors_df=pd.concat([df,df_test]).loc[:,regressors_list],
                                  periods=1, n_historic_predictions=31*1)

      forecast_test = m.predict(future)
      f31d=forecast_test.loc[forecast_test.ds>'2018-12-31','ds':'trend']
      f31d.set_index('ds',inplace=True)
      f31d.rename(columns={'yhat1':'yhatn'},inplace=True)

    else: # rolling the model forward 31 days on the first day
      future = m.make_future_dataframe(m.create_df_with_events(pd.concat([df]),df_hol), events_df=df_hol,
                                regressors_df=pd.concat([df,df_test]).loc[:,regressors_list],
                                periods=31, n_historic_predictions=31*1)

      forecast_test = m.predict(future,raw=True)
      f31d=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and '%' not in c]].T
      f31_5=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and ' 5.0%' in c]].T
      f31_95=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and ' 95.0%' in c]].T

      # f31d=forecast_test.loc[forecast_test.ds=='2018-12-31',[f"yhat{i}" for i in range(1,32)]].T
      # f31_5=forecast_test.loc[forecast_test.ds=='2018-12-31',[c for c in forecast_test.columns if ' 5.0%' in c]].T
      # f31_95=forecast_test.loc[forecast_test.ds=='2018-12-31',[c for c in forecast_test.columns if ' 95.0%' in c]].T

      f31d['yhat upper']=f31_95.values
      f31d['yhat lower']=f31_5.values
      f31d['ds']=df_test.ds.values
      f31d.set_index('ds',inplace=True)
      f31d.rename(columns={f31d.columns[0]:'yhatn'},inplace=True)
      f31d['y']=df_test.y.values

    f31d.to_pickle(fpathm+f"/{stn}_model_{modelnr}_f31d.pkl")

    # Plot 31d forecast vs. test data
    fig,ax=plt_mth(f31d.rename(columns={'yhat1 5.0%':'yhat lower','yhat1 95.0%':'yhat upper'}),stn,modelnr)
    fig.savefig(fpathm+f"/{stn}_{modelnr}_31d_sales_forecast.png",bbox_inches='tight')

    print(f"Model {modelnr}: {stn} 31d forecast {f31d.yhatn.sum():.2f} vs. test data {f31d.y.sum():.2f}.  Error: {-(f31d.y.sum()-f31d.yhatn.sum())/f31d.y.sum()*100:.1f}%")

    plt.close()

## 8. Model 7 with added PVP deviation of nearest station

In [ ]:
dict_M={}
valid=False
modelnr=8
fit_model=False


fpathm = fpath+f"/m31_{modelnr}"
if not os.path.exists(fpathm):
    os.mkdir(fpathm)

for stn in tqdm([f"ES{nr}_{pr}" for nr in range(1,13) for pr in ["95","GOA"]]):

    # add extra holidays/events
    # with open(fpath+'/df_holi_extra.pkl', 'rb') as f:
    #     dict_hol_extra=pickle.load(f)

    if stn[:2]=='ES' and stn[-3:]=="GOA":
        df_hol_extra=dict_hol_extra[f"df_holi_extra_{stn}"]
        df_hol_extra=pd.concat([df_hol_extra,pd.DataFrame({'ds':[datetime.datetime(2019,1,4)],'event':['Ev01']},index=[0])],ignore_index=True)
        df_hol=pd.concat([df_hol_base,df_hol_extra],ignore_index=True)
        print(f"Added {df_hol_extra.shape[0]} extra holidays for",stn)
        print(df_hol_extra)
    else:
        df_hol=df_hol_base

    df=df_v_m.loc[:,[stn]].reset_index().rename(columns={'sale_date':'ds',stn:'y'})
    df_test=df_v_m_test.loc[:,[stn]].reset_index().rename(columns={'sale_date':'ds',stn:'y'})

    # add lagged regressor with difference of y
    # do the difference on concatenated df and df_test for continuity
    lagg_I=pd.concat([df,df_test],ignore_index=True).y.diff().fillna(0)

    df["I"] = lagg_I.iloc[:len(df)].values
    df_test["I"]=lagg_I.iloc[len(df):].values

    # add weather data, temp, precipitation, wind
    # df["tmax"] = df_v_m.tmax.values
    df["tmed"] = df_v_m.tmed.values
    # df["tmin"] = df_v_m.tmin.values
    df["prec"] = df_v_m.prec.values
    df["velmedia"] = df_v_m.velmedia.values
    df["sol"] = df_v_m.sol.values
    df["racha"] = df_v_m.racha.values

    # add price deviation from median for this station
    df["pvpdev"]=(df_p_m[stn]-df_p_m["MED_"+stn.split('_')[1]]).values
    df["pvpdev_n"]=(df_p_m[stn_near.loc[stn.split('_')[0]].Nearest+'_'+stn.split('_')[1]]\
                  -df_p_m["MED_"+stn.split('_')[1]]).values
    # df_test["tmax"] = df_v_m_test.tmax.values
    df_test["tmed"] = df_v_m_test.tmax.values
    # df_test["tmin"] = df_v_m_test.tmax.values
    df_test["prec"] = df_v_m_test.prec.values
    df_test["velmedia"] = df_v_m_test.velmedia.values
    df_test["sol"] = df_v_m_test.sol.values
    df_test["racha"] = df_v_m_test.racha.values

    # add price deviation from median for this station
    df_test["pvpdev"]=(df_p_m_test[stn]-df_p_m_test["MED_"+stn.split('_')[1]]).values
    df_test["pvpdev_n"]=(df_p_m_test[stn_near.loc[stn.split('_')[0]].Nearest+'_'+stn.split('_')[1]]\
                  -df_p_m_test["MED_"+stn.split('_')[1]]).values

    m=NeuralProphet(
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=False,
                    seasonality_mode="additive",
                    n_lags=7,
                    ar_reg=0.1,
                    n_forecasts=31,
                    quantiles=[0.05,0.95],
                    #epochs=60,
                    )

    # add holidays
    for e in df_hol.event.unique():
      m.add_events(e,regularization=None,mode='additive')#,lower_window=0,upper_window=1)

    # add lagged regressor
    m.add_lagged_regressor("I",normalize="standardize",n_lags=7,regularization=0.1)
    m.add_lagged_regressor("pvpdev",normalize="standardize")
    m.add_lagged_regressor("pvpdev_n",normalize="standardize") #can add lags later

    # add weather data as future regressors and assume reliable weather forecast is available for 1 day ahead
    # m.add_future_regressor("tmax",normalize="standardize")
    m.add_future_regressor("tmed",normalize="standardize")
    # m.add_future_regressor("tmin",normalize="standardize")
    m.add_future_regressor("prec",normalize="standardize")
    m.add_future_regressor("velmedia",normalize="standardize")
    m.add_future_regressor("sol",normalize="standardize")
    m.add_future_regressor("racha",normalize="standardize")

    df_new=m.create_df_with_events(df, df_hol)


    if fit_model:
        if valid:
            df_train, df_val = m.split_df(df_new, freq="D", valid_p=0.1)
            metrics=m.fit(df_train,freq='D',validation_df=df_val, progress='bar')
        else:
            df_train=df_new
            metrics=m.fit(df_train,freq='D', progress='bar')
        print(metrics.tail(1))

        # save fitted model to disk
        with open(fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl', "wb") as f:
            pickle.dump(m, f, pickle.HIGHEST_PROTOCOL)
    else:
        print("Loading model from file: "+fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl')
        with open(fpath+f'/models_31d/{stn}_model_{modelnr}_model.pkl', "rb") as f:
            m=pickle.load(f)


    m.set_plotting_backend("matplotlib")

    # Forecast test period
    regressors_list=['I','pvpdev','pvpdev_n','tmed','prec','velmedia','sol','racha']

    # one forecast on '2018-12-31' for 31 days ahead (2019-01-01 to 2019-01-31)
    # in seasonal forecasting the model does not need own forecast to roll the model forward
    # so we can use the same model to forecast the next 31 days else we ask the model to roll the model forward 31 days
    if m.model.n_forecasts==1:
      print("One forecast!")
      future = m.make_future_dataframe(m.create_df_with_events(pd.concat([df,df_test]),df_hol), events_df=df_hol,
                                  regressors_df=pd.concat([df,df_test]).loc[:,regressors_list],
                                  periods=1, n_historic_predictions=31*1)

      forecast_test = m.predict(future)
      f31d=forecast_test.loc[forecast_test.ds>'2018-12-31','ds':'trend']
      f31d.set_index('ds',inplace=True)
      f31d.rename(columns={'yhat1':'yhatn'},inplace=True)

    else: # rolling the model forward 31 days on the first day
      future = m.make_future_dataframe(m.create_df_with_events(pd.concat([df]),df_hol), events_df=df_hol,
                                regressors_df=pd.concat([df,df_test]).loc[:,regressors_list],
                                periods=31, n_historic_predictions=31*1)

      forecast_test = m.predict(future,raw=True)
      f31d=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and '%' not in c]].T
      f31_5=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and ' 5.0%' in c]].T
      f31_95=forecast_test.loc[forecast_test.ds=='2019-01-01',[c for c in forecast_test.columns if 'step' in c and ' 95.0%' in c]].T

      # f31d=forecast_test.loc[forecast_test.ds=='2018-12-31',[f"yhat{i}" for i in range(1,32)]].T
      # f31_5=forecast_test.loc[forecast_test.ds=='2018-12-31',[c for c in forecast_test.columns if ' 5.0%' in c]].T
      # f31_95=forecast_test.loc[forecast_test.ds=='2018-12-31',[c for c in forecast_test.columns if ' 95.0%' in c]].T

      f31d['yhat upper']=f31_95.values
      f31d['yhat lower']=f31_5.values
      f31d['ds']=df_test.ds.values
      f31d.set_index('ds',inplace=True)
      f31d.rename(columns={f31d.columns[0]:'yhatn'},inplace=True)
      f31d['y']=df_test.y.values

    f31d.to_pickle(fpathm+f"/{stn}_model_{modelnr}_f31d.pkl")

    # Plot 31d forecast vs. test data
    fig,ax=plt_mth(f31d.rename(columns={'yhat1 5.0%':'yhat lower','yhat1 95.0%':'yhat upper'}),stn,modelnr)
    fig.savefig(fpathm+f"/{stn}_{modelnr}_31d_sales_forecast.png",bbox_inches='tight')

    print(f"Model {modelnr}: {stn} 31d forecast {f31d.yhatn.sum():.2f} vs. test data {f31d.y.sum():.2f}.  Error: {-(f31d.y.sum()-f31d.yhatn.sum())/f31d.y.sum()*100:.1f}%")

    plt.close()